In [1]:
import pprint, itertools
import numpy as np
import pandas as pd
import scipy as sp

from sklearn import neighbors, metrics, model_selection
from matplotlib import pyplot as plt

In [2]:
survived_test = pd.read_csv('/home/abdel/Documents/Kaggle/kaggle-titanic/survived_test.csv', sep="|", index_col='PassengerId')
survived_train = pd.read_csv('/home/abdel/Documents/Kaggle/kaggle-titanic/survived_train.csv', sep="|", index_col='PassengerId')

test = pd.read_csv('/home/abdel/Documents/Kaggle/kaggle-titanic/data_test.csv', sep="|", index_col='PassengerId')
train = pd.read_csv('/home/abdel/Documents/Kaggle/kaggle-titanic/data_train.csv', sep="|", index_col='PassengerId')

In [3]:
train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Bridge,Cabin
PassengerId,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7,S,NaN,NaN
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71,C,C8,C85
3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7,S,NaN,NaN
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53,S,C1,C123
5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8,S,NaN,NaN


In [4]:
print '=============='
print 'Train is null'
print '=============='
for colonne in train.columns :
    if train[colonne].isnull().any() :
        print colonne
print '==============' 
print 'Test is null'
print '=============='
for colonne in test.columns :
    if test[colonne].isnull().any() :
        print colonne

Train is null
Age
Embarked
Bridge
Cabin
Test is null
Age
Fare
Bridge
Cabin


In [148]:
print '=============='
print 'survived_train is null'
print '=============='
if survived_train.isnull().any()[0] :
    print 'oui'
print '==============' 
print 'survived_test is null'
print '=============='
if survived_test.isnull().any()[0] :
    print 'oui'

survived_train is null
survived_test is null


In [150]:
train.shape[0], survived_train.shape[0]

(891, 891)

# <font color="red">Preprocess</font>

In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(['male', 'female'])

print le.classes_
print le.transform(train['Sex'])[:10] 
print (train['Sex'] == le.inverse_transform(le.transform(train['Sex']))).all()

new_train_sex = le.transform(train['Sex'])
new_test_sex = le.transform(test['Sex'])

['female' 'male']
[1 0 0 0 1 1 1 1 0 0]
True


In [6]:
train_index = train['Sex'].index
test_index = test['Sex'].index

train['Sex'] = pd.Series(new_train_sex, index=train_index)
test['Sex'] = pd.Series(new_test_sex, index=test_index)

train.head(), test.head()

(             Pclass                                               Name  Sex  \
 PassengerId                                                                   
 1                 3                            Braund, Mr. Owen Harris    1   
 2                 1  Cumings, Mrs. John Bradley (Florence Briggs Th...    0   
 3                 3                             Heikkinen, Miss. Laina    0   
 4                 1       Futrelle, Mrs. Jacques Heath (Lily May Peel)    0   
 5                 3                           Allen, Mr. William Henry    1   
 
               Age  SibSp  Parch            Ticket  Fare Embarked Bridge Cabin  
 PassengerId                                                                    
 1            22.0      1      0         A/5 21171     7        S    NaN   NaN  
 2            38.0      1      0          PC 17599    71        C     C8   C85  
 3            26.0      0      0  STON/O2. 3101282     7        S    NaN   NaN  
 4            35.0      1      0 

# <font color="red">Reconstitution des données manquantes</font>

## <font color="purple">train Âge with mean</font>

In [7]:
train['Age'].isnull().any()

True

In [8]:
train_age_missing = train['Age'][train['Age'].isnull()]
train_age_missing.index
train_age_missing.values[:5]

array([ nan,  nan,  nan,  nan,  nan])

In [9]:
train_age_given = train['Age'][train['Age'].notnull()]
train_age_given.index
train_age_given.describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [10]:
train_age_missing.shape, train_age_given.shape

((177,), (714,))

In [11]:
int(train_age_given.mean())

29

In [12]:
train['Age'] = train['Age'].fillna(int(train_age_given.mean()))

In [13]:
train['Age'].notnull().all()

True

In [14]:
(train['Age'].loc[train_age_given.index] == train_age_given).all()

True

In [15]:
(train['Age'].loc[train_age_missing.index]).unique()

array([ 29.])

## <font color="purple">test Âge with mean</font>

In [16]:
test['Age'].isnull().any()

True

In [17]:
test_age_missing = test['Age'][test['Age'].isnull()]
test_age_given = test['Age'][test['Age'].notnull()]

In [18]:
test_age_missing.shape, test_age_given.shape

((86,), (332,))

In [19]:
test_age_given.describe()

count    332.000000
mean      30.272590
std       14.181209
min        0.170000
25%       21.000000
50%       27.000000
75%       39.000000
max       76.000000
Name: Age, dtype: float64

In [20]:
test['Age'] = test['Age'].fillna(int(test_age_given.mean()))

In [21]:
test['Age'].notnull().all()

True

In [22]:
(test['Age'].loc[test_age_given.index] == test_age_given).all()

True

In [23]:
(test['Age'].loc[test_age_missing.index]).unique()

array([ 30.])

#### <font color="blue">Le train et le test ont leur colonne âge remplie avec leurs moyennes d'âge respectives</font>

### <font color="orange">Type = Int</font>

In [24]:
train['Age'] = train['Age'].astype('int')
test['Age'] = test['Age'].astype('int')

## <font color="purple">test Fare with mean</font>

In [25]:
test['Fare'].isnull().any()

True

In [26]:
test['Fare'][test['Fare'].isnull()]

PassengerId
1044   NaN
Name: Fare, dtype: float64

In [27]:
test['Fare'] = test['Fare'].fillna(test['Fare'][test['Fare'].notnull()].mean())

In [28]:
test['Fare'].notnull().all()

True

In [29]:
test['Fare'].loc[1044]

35.627188489208599

In [30]:
test['Fare'].mean()

35.6271884892086

## <font color="purple">train Embarked with most frequently value</font>

In [31]:
train['Embarked'].isnull().any()

True

In [32]:
train_embarked_missing = train['Embarked'][train['Embarked'].isnull()]
train_embarked_given = train['Embarked'][train['Embarked'].notnull()]

### <font color="brown">Remplir avec la valeur de plus grande fréquence</font>

In [33]:
train['Embarked'].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

In [34]:
train['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [35]:
train['Embarked'].value_counts().index

Index([u'S', u'C', u'Q'], dtype='object')

In [36]:
train['Embarked'].value_counts().index[0]

'S'

In [37]:
train['Embarked'].value_counts()/train['Embarked'].shape[0]

S    0.722783
C    0.188552
Q    0.086420
Name: Embarked, dtype: float64

In [38]:
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].value_counts().index[0])

In [39]:
train['Embarked'].notnull().all()

True

## <font color="purple">test Embarked with most frequently value</font>

In [40]:
test['Embarked'].isnull().any()

False

In [41]:
test_embarked_missing = test['Embarked'][test['Embarked'].isnull()]
test_embarked_given = test['Embarked'][test['Embarked'].notnull()]

### <font color="brown">Remplir avec la valeur de plus grande fréquence</font>

In [42]:
test['Embarked'].describe()

count     418
unique      3
top         S
freq      270
Name: Embarked, dtype: object

In [43]:
test['Embarked'].value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

In [44]:
test['Embarked'].value_counts().index

Index([u'S', u'C', u'Q'], dtype='object')

In [45]:
test['Embarked'].value_counts().index[0]

'S'

In [46]:
test['Embarked'].value_counts()/test['Embarked'].shape[0]

S    0.645933
C    0.244019
Q    0.110048
Name: Embarked, dtype: float64

In [47]:
test['Embarked'] = test['Embarked'].fillna(test['Embarked'].value_counts().index[0])

In [48]:
test['Embarked'].notnull().all()

True

#### <font color="blue">Le train et le test ont leur colonne Embarked remplie avec leurs moyennes d'âge respectives</font>

### <font color="brown">Preprocess Embarked</font>

In [49]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(['S', 'C', 'Q'])

print train['Embarked'].unique()
print le.classes_
print le.transform(train['Embarked'])[:10]
print le.transform(test['Embarked'])[:10] 
print (train['Embarked'] == le.inverse_transform(le.transform(train['Embarked']))).all()

new_train_embarked = le.transform(train['Embarked'])
new_test_embarked = le.transform(test['Embarked'])

['S' 'C' 'Q']
['C' 'Q' 'S']
[2 0 2 2 2 1 2 2 2 0]
[1 2 1 2 2 2 1 2 0 2]
True


In [50]:
train_index = train['Embarked'].index
train['Embarked'] = pd.Series(new_train_embarked, index=train_index)
train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Bridge,Cabin
PassengerId,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",1,22,1,0,A/5 21171,7,2,NaN,NaN
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38,1,0,PC 17599,71,0,C8,C85
3,3,"Heikkinen, Miss. Laina",0,26,0,0,STON/O2. 3101282,7,2,NaN,NaN
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35,1,0,113803,53,2,C1,C123
5,3,"Allen, Mr. William Henry",1,35,0,0,373450,8,2,NaN,NaN


In [51]:
test_index = test['Embarked'].index
test['Embarked'] = pd.Series(new_test_embarked, index=test_index)
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Bridge,Cabin
PassengerId,,,,,,,,,,,
892,3,"Kelly, Mr. James",1,34,0,0,330911,7.8292,1,NaN,NaN
893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47,1,0,363272,7.0000,2,NaN,NaN
894,2,"Myles, Mr. Thomas Francis",1,62,0,0,240276,9.6875,1,NaN,NaN
895,3,"Wirz, Mr. Albert",1,27,0,0,315154,8.6625,2,NaN,NaN
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22,1,1,3101298,12.2875,2,NaN,NaN


In [52]:
train[['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Embarked']].notnull().all().all()

True

In [53]:
test[['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Embarked']].notnull().all().all()

True

# <font color="red">Cabin Bridge Level</font>

#### <font color="blue">Ce qui permet de retrouver le numéro de cabine : Pclass, SibSip, Parch, Fare voir Age.</font>

## <font color="purple">train Cabin & train Bridge</font>

In [54]:
train['Cabin'].isnull().any()

True

In [55]:
train_cabin_missing = train['Cabin'][train['Cabin'].isnull()]
train_cabin_given = train['Cabin'][train['Cabin'].notnull()]

In [56]:
train['Cabin'].describe()

count             204
unique            147
top       C23 C25 C27
freq                4
Name: Cabin, dtype: object

In [57]:
train['Cabin'].unique()

array([nan, 'C85', 'C123', 'E46', 'G6', 'C103', 'D56', 'A6', 'C23 C25 C27',
       'B78', 'D33', 'B30', 'C52', 'B28', 'C83', 'F33', 'F G73', 'E31',
       'A5', 'D10 D12', 'D26', 'C110', 'B58 B60', 'E101', 'F E69', 'D47',
       'B86', 'F2', 'C2', 'E33', 'B19', 'A7', 'C49', 'F4', 'A32', 'B4',
       'B80', 'A31', 'D36', 'D15', 'C93', 'C78', 'D35', 'C87', 'B77',
       'E67', 'B94', 'C125', 'C99', 'C118', 'D7', 'A19', 'B49', 'D',
       'C22 C26', 'C106', 'C65', 'E36', 'C54', 'B57 B59 B63 B66', 'C7',
       'E34', 'C32', 'B18', 'C124', 'C91', 'E40', 'T', 'C128', 'D37',
       'B35', 'E50', 'C82', 'B96 B98', 'E10', 'E44', 'A34', 'C104', 'C111',
       'C92', 'E38', 'D21', 'E12', 'E63', 'A14', 'B37', 'C30', 'D20',
       'B79', 'E25', 'D46', 'B73', 'C95', 'B38', 'B39', 'B22', 'C86',
       'C70', 'A16', 'C101', 'C68', 'A10', 'E68', 'B41', 'A20', 'D19',
       'D50', 'D9', 'A23', 'B50', 'A26', 'D48', 'E58', 'C126', 'B71',
       'B51 B53 B55', 'D49', 'B5', 'B20', 'F G63', 'C62 C64', 'E24',

In [58]:
train['Bridge'].unique()

array([nan, 'C8', 'C1', 'E4', 'G6', 'D5', 'A6', 'C2', 'B7', 'D3', 'B3',
       'C5', 'B2', 'F3', 'F ', 'E3', 'A5', 'D1', 'D2', 'B5', 'E1', 'D4',
       'B8', 'F2', 'B1', 'A7', 'C4', 'F4', 'A3', 'B4', 'C9', 'C7', 'E6',
       'B9', 'D7', 'A1', 'D', 'C6', 'C3', 'T', 'E5', 'E2', 'A2', 'D9',
       'E8', 'E7', 'D6', 'B6'], dtype=object)

In [59]:
train_bridge_missing = train['Bridge'][train['Bridge'].isnull()]
train_bridge_given = train['Bridge'][train['Bridge'].notnull()]

In [60]:
index_train_bridge_given = train_bridge_given.index

In [61]:
train_bridge_given.unique()

array(['C8', 'C1', 'E4', 'G6', 'D5', 'A6', 'C2', 'B7', 'D3', 'B3', 'C5',
       'B2', 'F3', 'F ', 'E3', 'A5', 'D1', 'D2', 'B5', 'E1', 'D4', 'B8',
       'F2', 'B1', 'A7', 'C4', 'F4', 'A3', 'B4', 'C9', 'C7', 'E6', 'B9',
       'D7', 'A1', 'D', 'C6', 'C3', 'T', 'E5', 'E2', 'A2', 'D9', 'E8',
       'E7', 'D6', 'B6'], dtype=object)

## <font color="purple">test Cabin & test Bridge</font>

In [62]:
test['Cabin'].isnull().any()

True

In [63]:
test_cabin_missing = test['Cabin'][test['Cabin'].isnull()]
test_cabin_given = test['Cabin'][test['Cabin'].notnull()]

In [64]:
test['Cabin'].describe()

count                  91
unique                 76
top       B57 B59 B63 B66
freq                    3
Name: Cabin, dtype: object

In [65]:
test['Cabin'].unique()

array([nan, 'B45', 'E31', 'B57 B59 B63 B66', 'B36', 'A21', 'C78', 'D34',
       'D19', 'A9', 'D15', 'C31', 'C23 C25 C27', 'F G63', 'B61', 'C53',
       'D43', 'C130', 'C132', 'C101', 'C55 C57', 'B71', 'C46', 'C116', 'F',
       'A29', 'G6', 'C6', 'C28', 'C51', 'E46', 'C54', 'C97', 'D22', 'B10',
       'F4', 'E45', 'E52', 'D30', 'B58 B60', 'E34', 'C62 C64', 'A11',
       'B11', 'C80', 'F33', 'C85', 'D37', 'C86', 'D21', 'C89', 'F E46',
       'A34', 'D', 'B26', 'C22 C26', 'B69', 'C32', 'B78', 'F E57', 'F2',
       'A18', 'C106', 'B51 B53 B55', 'D10 D12', 'E60', 'E50', 'E39 E41',
       'B52 B54 B56', 'C39', 'B24', 'D28', 'B41', 'C7', 'D40', 'D38',
       'C105'], dtype=object)

In [66]:
test['Bridge'].unique()

array([nan, 'B4', 'E3', 'B5', 'B3', 'A2', 'C7', 'D3', 'D1', 'A9', 'C3',
       'C2', 'F ', 'B6', 'C5', 'D4', 'C1', 'B7', 'C4', 'F', 'G6', 'C6',
       'E4', 'C9', 'D2', 'B1', 'F4', 'E5', 'A1', 'C8', 'F3', 'A3', 'D',
       'B2', 'F2', 'E6'], dtype=object)

In [67]:
test['Bridge'].isnull().any()

True

In [68]:
test_bridge_missing = test['Bridge'][test['Bridge'].isnull()]
test_bridge_given = test['Bridge'][test['Bridge'].notnull()]

In [69]:
index_test_bridge_given = test_bridge_given.index

In [70]:
test_bridge_given.unique()

array(['B4', 'E3', 'B5', 'B3', 'A2', 'C7', 'D3', 'D1', 'A9', 'C3', 'C2',
       'F ', 'B6', 'C5', 'D4', 'C1', 'B7', 'C4', 'F', 'G6', 'C6', 'E4',
       'C9', 'D2', 'B1', 'F4', 'E5', 'A1', 'C8', 'F3', 'A3', 'D', 'B2',
       'F2', 'E6'], dtype=object)

## <font color="purple">Création de la colonne Level</font>

In [71]:
train['Level'] = train_bridge_given.str.slice(0,1)
test['Level'] = test_bridge_given.str.slice(0,1)

In [72]:
train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Bridge,Cabin,Level
PassengerId,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",1,22,1,0,A/5 21171,7,2,NaN,NaN,NaN
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38,1,0,PC 17599,71,0,C8,C85,C
3,3,"Heikkinen, Miss. Laina",0,26,0,0,STON/O2. 3101282,7,2,NaN,NaN,NaN
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35,1,0,113803,53,2,C1,C123,C
5,3,"Allen, Mr. William Henry",1,35,0,0,373450,8,2,NaN,NaN,NaN


In [73]:
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Bridge,Cabin,Level
PassengerId,,,,,,,,,,,,
892,3,"Kelly, Mr. James",1,34,0,0,330911,7.8292,1,NaN,NaN,NaN
893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47,1,0,363272,7.0000,2,NaN,NaN,NaN
894,2,"Myles, Mr. Thomas Francis",1,62,0,0,240276,9.6875,1,NaN,NaN,NaN
895,3,"Wirz, Mr. Albert",1,27,0,0,315154,8.6625,2,NaN,NaN,NaN
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22,1,1,3101298,12.2875,2,NaN,NaN,NaN


In [74]:
train['Level'].unique()

array([nan, 'C', 'E', 'G', 'D', 'A', 'B', 'F', 'T'], dtype=object)

In [75]:
test.to_csv('/home/abdel/Documents/Kaggle/kaggle-titanic/data_preprocess_fill_test.csv', sep="|")
train.to_csv('/home/abdel/Documents/Kaggle/kaggle-titanic/data_preprocess_fill_train.csv', sep="|")

In [76]:
test = pd.read_csv('/home/abdel/Documents/Kaggle/kaggle-titanic/data_preprocess_fill_test.csv', sep="|", index_col='PassengerId')
train = pd.read_csv('/home/abdel/Documents/Kaggle/kaggle-titanic/data_preprocess_fill_train.csv', sep="|", index_col='PassengerId')

### <font color="orange">Comparaison Pclass et Level</font>

In [77]:
train[train['Pclass'] == 1]['Level'].unique()

array(['C', 'E', 'A', nan, 'B', 'D', 'T'], dtype=object)

In [78]:
train[train['Pclass'] == 3]['Level'].unique()

array([nan, 'G', 'F', 'E'], dtype=object)

In [79]:
train[train['Pclass'] == 2]['Level'].unique()

array([nan, 'D', 'F', 'E'], dtype=object)

#### <font color="blue">Les Level A, B, C et T sont toutes en Pclass 1</font>
#### <font color="blue">Le Level D est en Pclass 1 et 3</font>
#### <font color="blue">Le Level E est dans toutes les Pclass</font>
#### <font color="blue">Le Level F est en Pclass 2 et 3</font>
#### <font color="blue">Le Level G est en Pclass 2</font>

#### <font color="blue">Pour les Levels A,B,C et G c'est facile</font>
#### <font color="blue">Pour les Levels D, E et F c'est plus compliqué, voyons si une Pclass est majoritaire !</font>

In [80]:
train[train['Level'].isnull()]['Pclass'].unique()

array([3, 2, 1])

In [81]:
train[train['Pclass'] == 1]['Level'].value_counts()

C    59
B    47
D    29
E    25
A    15
T     1
Name: Level, dtype: int64

In [82]:
train[train['Pclass'] == 2]['Level'].value_counts()

F    8
D    4
E    4
Name: Level, dtype: int64

In [83]:
train[train['Pclass'] == 3]['Level'].value_counts()

F    5
G    4
E    3
Name: Level, dtype: int64

### <font color="orange">Comparaison Pclass et SibSip</font>

In [84]:
train['SibSp'].unique()

array([1, 0, 3, 4, 2, 5, 8])

In [85]:
train[train['Level'].isnull()]['SibSp'].unique()

array([1, 0, 3, 4, 2, 5, 8])

#### <font color="blue">Tous les SibSp contiennent des nan pour le Level</font>

In [86]:
for i in train['SibSp'].unique() :
    print i, train[train['SibSp'] == i]['Level'].unique() 

1 [nan 'C' 'G' 'B' 'D' 'E' 'F' 'A']
0 [nan 'E' 'C' 'D' 'A' 'B' 'F' 'G' 'T']
3 [nan 'C']
4 [nan]
2 [nan 'F' 'C' 'B']
5 [nan]
8 [nan]


In [87]:
for i in train['SibSp'].unique() :
    print i, train[train['SibSp'] == i]['Level'].value_counts().index

1 Index([u'C', u'D', u'B', u'E', u'F', u'G', u'A'], dtype='object')
0 Index([u'B', u'C', u'E', u'D', u'A', u'F', u'G', u'T'], dtype='object')
3 Index([u'C'], dtype='object')
4 Index([], dtype='object')
2 Index([u'F', u'C', u'B'], dtype='object')
5 Index([], dtype='object')
8 Index([], dtype='object')


#### <font color="blue">Pour les SibSp égaux à 4, 5, 8, il n y a pas d'informations sur le Level</font>
#### <font color="blue">Pour les SibSp égale à 3, le Level est C</font>

### <font color="orange">Comparaison Pclass et Parch</font>

In [88]:
train['Parch'].unique()

array([0, 1, 2, 5, 3, 4, 6])

In [89]:
train[train['Level'].isnull()]['Parch'].unique()

array([0, 1, 2, 5, 3, 4, 6])

#### <font color="blue">Tous les Parch contiennent des nan pour le Level</font>

In [90]:
for i in train['Parch'].unique() :
    print i, train[train['Parch'] == i]['Level'].unique() 

0 [nan 'C' 'E' 'D' 'A' 'B' 'F' 'T']
1 [nan 'G' 'B' 'D' 'F' 'E' 'C']
2 [nan 'C' 'D' 'F' 'B' 'G' 'A' 'E']
5 [nan]
3 [nan]
4 [nan 'C']
6 [nan]


In [91]:
for i in train['Parch'].unique() :
    print i, train[train['Parch'] == i]['Level'].value_counts().index

0 Index([u'C', u'B', u'D', u'E', u'A', u'F', u'T'], dtype='object')
1 Index([u'B', u'E', u'D', u'C', u'F', u'G'], dtype='object')
2 Index([u'C', u'B', u'D', u'F', u'G', u'E', u'A'], dtype='object')
5 Index([], dtype='object')
3 Index([], dtype='object')
4 Index([u'C'], dtype='object')
6 Index([], dtype='object')


#### <font color="blue">Pour les Parch égaux à 0, 1, 2, il n y a pas d'informations sur le Level</font>
#### <font color="blue">Pour les Parch égale à 4, le Level est C</font>

### <font color="orange">Comparaison Pclass et Fare</font>

In [92]:
train['Fare'].unique()

array([  7,  71,  53,   8,  51,  21,  11,  30,  16,  26,  31,  29,  13,
        18,  35, 263,  27, 146,  10,  82,  52,   9,  41,  15,  17,  39,
        76,  61,  46,  80,  83,  73,  14,  56,  12,  47,  34,  20,  63,
        23,  77,  24, 247,  22,   6,  79,  36,  66,  69,  55,  25,  33,
        28,   0,  50, 113,  90,  86, 512, 153, 135,  19,  78,  91, 151,
       110, 108, 262, 164, 134,  57, 133,  75, 211,   4, 227, 120,  32,
        81,  89,  38,  49,  59,  93, 221, 106,  40,  42,  65,  37,   5])

In [93]:
train[train['Level'].isnull()]['Fare'].unique()

array([  7,   8,  21,  11,  30,  31,  16,  29,  13,  18,  26,  27,  10,
        82,  52,   9,  41,  15,  17,  39,  46,  73,  14,  56,  12,  47,
        34,  20,  23,  24,   6,  36,  61,  69,  25,   0,  22,  79, 512,
        19,  78, 110, 133, 135,   4, 227,  32,  49,  59, 106,  33,  42,
        65, 151,  37, 164])

#### <font color="blue">Tous les Fare contiennent des nan pour le Level</font>

In [94]:
for i in train['Fare'].unique() :
    print i, train[train['Fare'] == i]['Level'].unique() 

7 [nan 'F']
71 ['C' 'B']
53 ['C' 'E' 'D']
8 [nan 'E']
51 ['E' 'D' 'C']
21 [nan]
11 [nan]
30 [nan 'A' 'C' 'B' 'D']
16 ['G' nan]
26 ['C' nan 'D' 'F' 'A' 'E' 'B']
31 [nan 'A']
29 [nan 'C' 'B']
13 [nan 'D' 'E' 'F']
18 [nan]
35 ['A' 'C' 'T']
263 ['C']
27 [nan 'B' 'C']
146 ['B']
10 [nan 'F' 'G' 'E']
82 [nan]
52 [nan 'C' 'D' 'A' 'B']
9 [nan]
41 [nan]
15 [nan]
17 [nan]
39 [nan 'F' 'A' 'D']
76 ['D']
61 ['B' 'E' nan]
46 [nan]
80 ['B']
83 ['C' 'E']
73 [nan]
14 [nan]
56 [nan 'E' 'A']
12 [nan 'D' 'E']
47 [nan]
34 [nan 'A' 'D']
20 [nan]
63 ['D']
23 [nan]
77 ['D']
24 [nan]
247 ['B']
22 ['F' nan]
6 [nan]
79 ['B' nan 'E']
36 [nan]
66 ['C']
69 [nan 'B']
55 ['E']
25 [nan 'E' 'D']
33 ['B' nan]
28 ['C']
0 [nan 'B' 'A']
50 ['A']
113 ['D']
90 ['C']
86 ['B']
512 [nan 'B']
153 ['C']
135 ['C' nan]
19 [nan]
78 [nan 'D' 'C']
91 ['B']
151 ['C' nan]
110 [nan 'C']
108 ['C']
262 ['B']
164 ['C' nan]
134 ['E']
57 ['B']
133 [nan]
75 ['D']
211 ['C' 'B']
4 [nan]
227 [nan 'C']
120 ['B']
32 [nan 'D']
81 ['A']
89 ['C']
38 ['

In [95]:
for i in train['Fare'].unique() :
    print i, train[train['Fare'] == i]['Level'].value_counts().index

7 Index([u'F'], dtype='object')
71 Index([u'B', u'C'], dtype='object')
53 Index([u'C', u'E', u'D'], dtype='object')
8 Index([u'E'], dtype='object')
51 Index([u'C', u'D', u'E'], dtype='object')
21 Index([], dtype='object')
11 Index([], dtype='object')
30 Index([u'C', u'D', u'B', u'A'], dtype='object')
16 Index([u'G'], dtype='object')
26 Index([u'E', u'C', u'F', u'D', u'B', u'A'], dtype='object')
31 Index([u'A'], dtype='object')
29 Index([u'C', u'B'], dtype='object')
13 Index([u'D', u'F', u'E'], dtype='object')
18 Index([], dtype='object')
35 Index([u'C', u'A', u'T'], dtype='object')
263 Index([u'C'], dtype='object')
27 Index([u'B', u'C'], dtype='object')
146 Index([u'B'], dtype='object')
10 Index([u'F', u'G', u'E'], dtype='object')
82 Index([], dtype='object')
52 Index([u'D', u'C', u'B', u'A'], dtype='object')
9 Index([], dtype='object')
41 Index([], dtype='object')
15 Index([], dtype='object')
17 Index([], dtype='object')
39 Index([u'F', u'D', u'A'], dtype='object')
76 Index([u'D'], dt

#### <font color="blue">Pour les Fare égaux à 0, 1, 2, il n y a pas d'informations sur le Level</font>
#### <font color="blue">Pour les Fare égale à 4, le Level est C</font>

In [96]:
train[train['SibSp'] == 3].index

Int64Index([8, 25, 28, 64, 86, 89, 177, 230, 342, 375, 410, 486, 635, 643, 727,
            820],
           dtype='int64', name=u'PassengerId')

In [97]:
train[train['Parch'] == 4].index

Int64Index([168, 361, 439, 568], dtype='int64', name=u'PassengerId')

### <font color="orange">Prédiction du Train Level</font>

In [98]:
train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Bridge,Cabin,Level
PassengerId,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",1,22,1,0,A/5 21171,7,2,NaN,NaN,NaN
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38,1,0,PC 17599,71,0,C8,C85,C
3,3,"Heikkinen, Miss. Laina",0,26,0,0,STON/O2. 3101282,7,2,NaN,NaN,NaN
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35,1,0,113803,53,2,C1,C123,C
5,3,"Allen, Mr. William Henry",1,35,0,0,373450,8,2,NaN,NaN,NaN


In [99]:
train[['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Embarked']].notnull().all().all()

True

In [100]:
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Bridge,Cabin,Level
PassengerId,,,,,,,,,,,,
892,3,"Kelly, Mr. James",1,34,0,0,330911,7.8292,1,NaN,NaN,NaN
893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47,1,0,363272,7.0000,2,NaN,NaN,NaN
894,2,"Myles, Mr. Thomas Francis",1,62,0,0,240276,9.6875,1,NaN,NaN,NaN
895,3,"Wirz, Mr. Albert",1,27,0,0,315154,8.6625,2,NaN,NaN,NaN
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22,1,1,3101298,12.2875,2,NaN,NaN,NaN


In [101]:
test[['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Embarked']].notnull().all().all()

True

In [102]:
train_without_level = train.drop(['Cabin', 'Bridge', 'Level', 'Ticket', 'Name'], axis=1)
train_level = train['Level']

In [103]:
train_notnull_without_level = train_without_level[train_level.notnull()]
train_isnull_without_level = train_without_level[train_level.isnull()]

In [104]:
train_notnull_level = train_level[train_level.notnull()]
train_isnull_level = train_level[train_level.isnull()]

In [105]:
from sklearn import neighbors, metrics, model_selection
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_notnull_without_level, train_notnull_level)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [106]:
train_predict_level = neigh.predict(train_isnull_without_level)

In [107]:
train_predict_level[0:10]

array(['F', 'F', 'B', 'F', 'F', 'E', 'D', 'F', 'A', 'D'], dtype=object)

In [108]:
type(train_predict_level)

numpy.ndarray

In [109]:
train_isnull_level.head()

PassengerId
1    NaN
3    NaN
5    NaN
6    NaN
8    NaN
Name: Level, dtype: object

In [110]:
type(train_isnull_level)

pandas.core.series.Series

In [111]:
train_isnull_level.values[0:10]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], dtype=object)

In [112]:
train_isnull_level = pd.Series(data=train_predict_level, index=train_isnull_level.index)

In [113]:
for i in train_isnull_level.index :
    train['Level'].loc[i] = train_isnull_level.loc[i]

/home/abdel/.local/lib/python2.7/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [114]:
train['Level'].notnull().all()

True

### <font color="orange">Prédiction du Train Bridge</font>

In [115]:
train_without_bridge = train.drop(['Cabin', 'Bridge', 'Level', 'Ticket', 'Name'], axis=1)
train_bridge = train['Bridge']

train_notnull_without_bridge = train_without_bridge[train_bridge.notnull()]
train_isnull_without_bridge = train_without_bridge[train_bridge.isnull()]

train_notnull_bridge = train_bridge[train_bridge.notnull()]
train_isnull_bridge = train_bridge[train_bridge.isnull()]

from sklearn import neighbors, metrics, model_selection
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_notnull_without_bridge, train_notnull_bridge)

train_predict_bridge = neigh.predict(train_isnull_without_bridge)
train_isnull_bridge = pd.Series(data=train_predict_bridge, index=train_isnull_bridge.index)

for i in train_isnull_bridge.index :
    train['Bridge'].loc[i] = train_isnull_bridge.loc[i]
    
train['Bridge'].notnull().all()

True

In [116]:
(train['Bridge'].str.slice(0,1) == train['Level']).value_counts()

True     766
False    125
dtype: int64

### <font color="orange">Prédiction du Train Cabin</font>

In [117]:
train_without_cabin = train.drop(['Cabin', 'Bridge', 'Level', 'Ticket', 'Name'], axis=1)
train_cabin = train['Cabin']

train_notnull_without_cabin = train_without_cabin[train_cabin.notnull()]
train_isnull_without_cabin = train_without_cabin[train_cabin.isnull()]

train_notnull_cabin = train_cabin[train_cabin.notnull()]
train_isnull_cabin = train_cabin[train_cabin.isnull()]

from sklearn import neighbors, metrics, model_selection
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_notnull_without_cabin, train_notnull_cabin)

train_predict_cabin = neigh.predict(train_isnull_without_cabin)
train_isnull_cabin = pd.Series(data=train_predict_cabin, index=train_isnull_cabin.index)

train['Cabin'].loc[train_isnull_cabin.index] = train_isnull_cabin
train['Cabin'].notnull().all()

True

In [118]:
(train['Cabin'].str.slice(0,2) == train['Bridge']).value_counts()

True     821
False     70
dtype: int64

In [119]:
(train['Cabin'].str.slice(0,1) == train['Level']).value_counts()

True     696
False    195
dtype: int64

#### <font color="blue">Les prédictions des Cabin, Bridge, Level coincident à 80%.</font>
#### <font color="blue">On va donc d'abord prédire les Cabin puis en déduire le Bridge et le Level.</font>

In [120]:
train['Bridge'] = train['Cabin'].str.slice(0,2)

In [121]:
train['Level'] = train['Cabin'].str.slice(0,1)

In [122]:
(train['Cabin'].str.slice(0,2) == train['Bridge']).value_counts()

True    891
dtype: int64

In [123]:
(train['Cabin'].str.slice(0,1) == train['Level']).value_counts()

True    891
dtype: int64

### <font color="orange">Prédiction du Test Cabin</font>

In [124]:
test_without_cabin = test.drop(['Cabin', 'Bridge', 'Level', 'Ticket', 'Name'], axis=1)
test_cabin = test['Cabin']

test_notnull_without_cabin = test_without_cabin[test_cabin.notnull()]
test_isnull_without_cabin = test_without_cabin[test_cabin.isnull()]

test_notnull_cabin = test_cabin[test_cabin.notnull()]
test_isnull_cabin = test_cabin[test_cabin.isnull()]

In [125]:
from sklearn import neighbors, metrics, model_selection
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(test_notnull_without_cabin, test_notnull_cabin)

test_predict_cabin = neigh.predict(test_isnull_without_cabin)
test_isnull_cabin = pd.Series(data=test_predict_cabin, index=test_isnull_cabin.index)

test['Cabin'].loc[test_isnull_cabin.index] = test_isnull_cabin
test['Cabin'].notnull().all()

True

In [126]:
(test['Cabin'].str.slice(0,2) == test['Bridge']).value_counts()

False    327
True      91
dtype: int64

In [127]:
(test['Cabin'].str.slice(0,1) == test['Level']).value_counts()

False    327
True      91
dtype: int64

In [128]:
test['Bridge'] = test['Cabin'].str.slice(0,2)

In [129]:
test['Level'] = test['Cabin'].str.slice(0,1)

# <font color="red">Name</font>

In [130]:
train['Name'].str.split(pat=',').head()

PassengerId
1                           [Braund,  Mr. Owen Harris]
2    [Cumings,  Mrs. John Bradley (Florence Briggs ...
3                            [Heikkinen,  Miss. Laina]
4      [Futrelle,  Mrs. Jacques Heath (Lily May Peel)]
5                          [Allen,  Mr. William Henry]
Name: Name, dtype: object

In [131]:
train['Nickname'] = np.nan

In [132]:
for i in train['Name'].index :
    train['Nickname'].loc[i] = train['Name'].str.split(pat=',').loc[i][0]

In [133]:
train['Nickname'].head()

PassengerId
1       Braund
2      Cumings
3    Heikkinen
4     Futrelle
5        Allen
Name: Nickname, dtype: object

In [134]:
train['Nickname'].value_counts()

Andersson      9
Sage           7
Carter         6
Goodwin        6
Skoog          6
Panula         6
Johnson        6
Rice           5
Lefebre        4
Gustafsson     4
Asplund        4
Ford           4
Smith          4
Palsson        4
Brown          4
Hart           4
Fortune        4
Harris         4
Williams       4
Harper         4
Baclini        4
Kelly          4
Graham         3
Elias          3
Meyer          3
Newell         3
Collyer        3
Boulos         3
Olsen          3
Goldsmith      3
              ..
Ostby          1
Cumings        1
Weisz          1
Sjoblom        1
O'Dwyer        1
McEvoy         1
Myhrman        1
Phillips       1
Rothes         1
Daniel         1
Horgan         1
Richard        1
Mitkoff        1
Salkjelsvik    1
Endres         1
Porter         1
Mellors        1
Caram          1
de Mulder      1
Angle          1
Slayter        1
Beesley        1
Dahl           1
Thorne         1
Lindell        1
Najib          1
Leonard        1
Gronnestad    

In [135]:
train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Bridge,Cabin,Level,Nickname
PassengerId,,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",1,22,1,0,A/5 21171,7,2,F,F G73,F,Braund
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38,1,0,PC 17599,71,0,C8,C85,C,Cumings
3,3,"Heikkinen, Miss. Laina",0,26,0,0,STON/O2. 3101282,7,2,E1,E101,E,Heikkinen
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35,1,0,113803,53,2,C1,C123,C,Futrelle
5,3,"Allen, Mr. William Henry",1,35,0,0,373450,8,2,B5,B51 B53 B55,B,Allen


In [136]:
train_level = train['Level']
train.drop(labels=['Level'], axis=1, inplace = True)
train.insert(9, 'Level', train_level)
train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Level,Bridge,Cabin,Nickname
PassengerId,,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",1,22,1,0,A/5 21171,7,2,F,F,F G73,Braund
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38,1,0,PC 17599,71,0,C,C8,C85,Cumings
3,3,"Heikkinen, Miss. Laina",0,26,0,0,STON/O2. 3101282,7,2,E,E1,E101,Heikkinen
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35,1,0,113803,53,2,C,C1,C123,Futrelle
5,3,"Allen, Mr. William Henry",1,35,0,0,373450,8,2,B,B5,B51 B53 B55,Allen


In [137]:
train_nickname = train['Nickname']
train.drop(labels=['Nickname'], axis=1, inplace = True)
train.insert(2, 'Nickname', train_nickname)
train.head()

,Pclass,Name,Nickname,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Level,Bridge,Cabin
PassengerId,,,,,,,,,,,,,
1,3,"Braund, Mr. Owen Harris",Braund,1,22,1,0,A/5 21171,7,2,F,F,F G73
2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",Cumings,0,38,1,0,PC 17599,71,0,C,C8,C85
3,3,"Heikkinen, Miss. Laina",Heikkinen,0,26,0,0,STON/O2. 3101282,7,2,E,E1,E101
4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",Futrelle,0,35,1,0,113803,53,2,C,C1,C123
5,3,"Allen, Mr. William Henry",Allen,1,35,0,0,373450,8,2,B,B5,B51 B53 B55


In [138]:
test['Nickname'] = np.nan
for i in test['Name'].index :
    test['Nickname'].loc[i] = test['Name'].str.split(pat=',').loc[i][0]

In [139]:
test_level = test['Level']
test.drop(labels=['Level'], axis=1, inplace = True)
test.insert(9, 'Level', test_level)
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Level,Bridge,Cabin,Nickname
PassengerId,,,,,,,,,,,,,
892,3,"Kelly, Mr. James",1,34,0,0,330911,7.8292,1,D,D,D,Kelly
893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47,1,0,363272,7.0000,2,B,B5,B52 B54 B56,Wilkes
894,2,"Myles, Mr. Thomas Francis",1,62,0,0,240276,9.6875,1,B,B2,B26,Myles
895,3,"Wirz, Mr. Albert",1,27,0,0,315154,8.6625,2,F,F,F E46,Wirz
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",0,22,1,1,3101298,12.2875,2,D,D3,D38,Hirvonen


In [140]:
test_nickname = test['Nickname']
test.drop(labels=['Nickname'], axis=1, inplace = True)
test.insert(2, 'Nickname', test_nickname)
test.head()

,Pclass,Name,Nickname,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Level,Bridge,Cabin
PassengerId,,,,,,,,,,,,,
892,3,"Kelly, Mr. James",Kelly,1,34,0,0,330911,7.8292,1,D,D,D
893,3,"Wilkes, Mrs. James (Ellen Needs)",Wilkes,0,47,1,0,363272,7.0000,2,B,B5,B52 B54 B56
894,2,"Myles, Mr. Thomas Francis",Myles,1,62,0,0,240276,9.6875,1,B,B2,B26
895,3,"Wirz, Mr. Albert",Wirz,1,27,0,0,315154,8.6625,2,F,F,F E46
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",Hirvonen,0,22,1,1,3101298,12.2875,2,D,D3,D38


In [141]:
print '=============='
print 'Train is null'
print '=============='
for colonne in train.columns :
    if train[colonne].isnull().any() :
        print colonne
print '==============' 
print 'Test is null'
print '=============='
for colonne in test.columns :
    if test[colonne].isnull().any() :
        print colonne

Train is null
Test is null


In [151]:
train.shape[0], survived_train.shape[0]

(891, 891)

In [149]:
test.to_csv('/home/abdel/Documents/Kaggle/kaggle-titanic/data_preprocess_fill_test.csv', sep="|")
train.to_csv('/home/abdel/Documents/Kaggle/kaggle-titanic/data_preprocess_fill_train.csv', sep="|")